## Get the Metadata for the 990 IRS files

In [ ]:
import os, sys
from joblib import Parallel, delayed

# METHOD 1
#import import_ipynb
#from api.classy990downloads import *

# METHOD 2 - This will reload without restarting notebook (debugging)
%run ../api/classy990downloads.ipynb

In [ ]:
#Set the YEAR of IRS 990 data that we want to download

year = 2017

In [ ]:
# Load the object that parses the IRS 990 index and will provide
# each individual form to us

#documents = classy990forms(config='validElements.conf', mongo='mongodb://mongo/irs990', debug=False)
documents = classy990forms(mongo='mongodb://mongo/irs990', dumpmongo=False, debug=True)
documents.year( year )
documents

In [ ]:
def readDoc(x):
    try:
        sys.stdout.write('Dowload EIN:%s\r' % x.index)
        sys.stdout.flush() 
        x.to_mongo()
    except Exception as e:
        print('oops: %s' % e)

In [ ]:
#
# THIS WILL ALWAYS RUN SINGLETHREAD SINCE OUR OBJECT NEVER RELEASES THE GIL.
#

# Loop over all forms and upload to the database.
num_cores=6
#Parallel(n_jobs=num_cores,require='sharedmem')(delayed(readDoc)(x) for x in documents)
Parallel(n_jobs=num_cores,backend='threading')(delayed(readDoc)(x) for x in documents)